# OpenStreetMap项目

### 选择区域
北京市，纬度39.7942到40.0408 经度116.1535到116.6046
### 区域地图链接
https://www.openstreetmap.org/export#map=10/39.9066/116.3905

选择北京市的原因在于北京是中国的首都，被全球最权威的世界城市研究机构之一GaWC评为世界一线城市，也是首批国家历史文化名城和世界上拥有世界文化遗产数最多的城市，我很喜欢北京。

### 文件大小

In [4]:
import os 

size = os.path.getsize("/Users/user/beijingmap.osm")
print 'beijingmap.osm:  %0.2f MB'% (size/1024.0/1024.0) 

beijingmap.osm:  92.36 MB


In [ ]:
ops.db ................ 56 MB
nodes.csv ............. 32.9 MB
nodes_tags.csv ........ 2 MB
ways.csv .............. 4.3 MB
ways_tags.csv ......... 5 MB
ways_nodes.cv ......... 11.5 MB 

### 项目中遇到的问题
街道名称比较乱:

1、大街(Avenue)有“Ave”，有“ave.”来表示的。

2、街(Jie)有“jie”来表示的，街道(Street)有“St”，有“Str”，还有“street”来表示的。

3、道路(Road)有“Rd”、“Rd,”、“Rd.”、“road”多种表示法，既不统一也不规范。

### 街道名称问题
将规范的街道名称提取出来，按照预期处理

In [ ]:
for _, street in st_types.iteritems():    
    for name in street:
        if street_type_re.search(name).group() in mapping.keys():              
            better_name = name.replace(street_type_re.search(name).group(), 
                    mapping[street_type_re.search(name).group()])    
            streets_repair[street_id(name)] = better_name
            print name, "->", better_name

如下所示，经过处理后，不规范的街道名称变得统一

In [ ]:
Yang Shan Rd, -> Yang Shan Road
chaoyang -> Chaoyang
19 Ritan. Bei Lu, Chaoyang district -> 19 Ritan. Bei Lu, Chaoyang District
east chang ave. -> east chang Avenue
Dongzhimen Outer St, Chaoyang, -> Dongzhimen Outer St, Chaoyang
North St -> North Street
Chongwenmen Wai St -> Chongwenmen Wai Street
Tie shu xie jie -> Tie shu xie Jie
W. Dazhalan Str -> W. Dazhalan Street
Xizhaosi Str -> Xizhaosi Street
Guozijian Str -> Guozijian Street
Maliandao Rd. -> Maliandao Road
Jiangtai road -> Jiangtai Road
B2 jiangtai road -> B2 jiangtai Road

## 清洗之后的数据导入SQL数据库


### 查看唯一用户的数量

In [ ]:
sqlite> SELECT count(*)
   ...> FROM (
   ...> SELECT nodes.user, COUNT(*) as num
   ...> FROM nodes,ways_nodes,ways
   ...> WHERE nodes.id = ways_nodes.node_id and ways_nodes.id = ways.id
   ...> GROUP BY nodes.uid) as fe;

1000

### 贡献地图的人top10:

In [ ]:
sqlite> SELECT nodes.user, COUNT(*) as num
   ...> FROM nodes
   ...> JOIN ways_nodes
   ...> ON nodes.id = ways_nodes.node_id
   ...> JOIN ways
   ...> ON ways_nodes.id = ways.id
   ...> GROUP BY nodes.uid
   ...> ORDER BY num DESC
   ...> LIMIT 10;

输出结果：
R438,98624
"Алекс Мок",67644
hanchao,42239
"Chen Jia",35029
m17design,18628
nuklearerWintersturm,18407
RationalTangle,12800
Esperanza36,11306
Popolon,7270
MrBoot,5502

贡献最多的人为“R438”，共98624条记录

### 查看节点数量

In [ ]:
SELECT COUNT(*) FROM nodes;

node节点数：390672


### 查看途径数量

In [ ]:
SELECT COUNT(*) FROM ways;

ways数：71277

### 查看北京建筑类型的数量

In [ ]:
sqlite> SELECT value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> WHERE key='amenity'
   ...> GROUP BY value
   ...> ORDER BY num DESC
   ...> LIMIT 10;

输出结果：
restaurant,1335
bank,422
fast_food,310
cafe,288
toilets,285
parking,165
bar,162
telephone,148
school,129
atm,101

结果显示所选北京区域的饭店最多，有1335家。学校有129所，可见北京对教育很重视，人口也比较多。

## 额外的数据探索
### 查看北京的道路情况

In [ ]:
sqlite> SELECT nodes_tags.value, count(*) as num
   ...> FROM ways_tags,ways,ways_nodes,nodes,nodes_tags
   ...> WHERE ways_tags.id = ways.id and ways.id = ways_nodes.id and 
ways_nodes.node_id = nodes.id and nodes.id = nodes_tags.id and nodes_tags.key = 'highway'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num desc
   ...> LIMIT 5;

输出结果：
traffic_signals,38693
bus_stop,12450
crossing,4883
motorway_junction,3412
platform,77

道路上最多的是交通信号灯，其次是公交车站和十字路口。

### 查看辐射最广的宗教(top3)

In [ ]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE
   ...> value='place_of_worship') dis
   ...> ON nodes_tags.id=dis.id
   ...> WHERE nodes_tags.key='religion'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC
   ...> LIMIT 3;

输出结果：
buddhist,8
christian,4
muslim,2

可以看到信佛教的人最多，其次是天主教，第三为穆斯林教。由于所选的区域比较小，关于宗教信仰的结果很少。

### 查看最受欢迎的菜系(top5)

In [ ]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
   ...> ON nodes_tags.id=i.id
   ...> WHERE nodes_tags.key='cuisine'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC
   ...> LIMIT 5;

输出结果：
chinese,168
japanese,19
italian,17
pizza;american,15
international,10

中国菜是最受欢迎的这一点毫无疑问，其次是日本料理，意大利菜，美式料理，国际菜

## 额外建议

项目所使用的数据集不完整，部分属性格式不统一，可以看到很多数据会有所缺失，要么属性没有，要么名称没有，对于精确度要求较高的话，openstreetmap数据集是达不到要求的，需要官方进一步去完善。

### 好处

如果数据集能得到完善，后期任何人在解析该数据集的时候会方便得多，也更容易从数据集中获得自己需要的有价值的信息。

### 预期问题

由于绘制openstreetmap的人形形色色，每个人的想法、习惯不同，对地图的补充也是自发行为，很难在格式、标准、完整性等各个方面上做到统一。

## 参考资料

清洗osm数据并转换成csv部分引用Udacity课程中的代码(案例研究：OpenStreetMap数据[SQL] )